In [1]:
# add the src folder to the path
import sys
sys.path.append('../')

from src.search import grid_search_cv
from src.utils import load_cup, parse_results
from src.validation import kfold_cv

from src.network import Network
from src.activations import ReLU, Sigmoid
from src.losses import MeanSquaredError
from src.metrics import MeanEuclideanError
from src.regularizers import L1, L2
from src.callbacks import EarlyStopping

X, y, scaler = load_cup(validation=False, scale_outputs=True)
y.shape

(1194, 2, 1)

In [ ]:
#Coarse grained grid search
model = Network(9)
model.add_layer(24, ReLU())
model.add_layer(16, ReLU())
model.add_layer(2, Sigmoid())

gs_results = grid_search_cv(
    model, 
    x=X, 
    y=y, 
    n_folds=3,
    metric=MeanEuclideanError(), 
    loss=MeanSquaredError(), 
    eta=[1e-2, 1e-3, 1e-1],
    nesterov=[0.8, 0.9, 0.7, 0.5, 0.6],
    reg_type=[L2, L1],
    reg_val=[1e-7, 1e-8, 1e-6, 1e-5],
    epochs=1000,
    scaler=scaler,
    workers=8,
    verbose=False
)

In [ ]:
results_df = parse_results(gs_results)
results_df.to_csv('../result_gs/2416_coarse.csv')
results_df.sort_values(by='accuracy', ascending=True)

In [ ]:
#Fine grained grid search
model = Network(9)
model.add_layer(32, ReLU())
model.add_layer(16, ReLU())
model.add_layer(2, Sigmoid(), initializer='xavier')

gs_results = grid_search_cv(
    model, 
    x=X, 
    y=y, 
    n_folds=3,
    metric=MeanEuclideanError(), 
    loss=MeanSquaredError(), 
    eta=[1e-2, 3e-2, 5e-2, 7e-2, 9e-2],
    nesterov=[0.55, 0.57, 0.59, 0.61, 0.63, 0.65],
    reg_type=[L2],
    reg_val=[1e-7, 1e-8, 1e-6, 1e-5],
    epochs=1000,
    scaler=scaler,
    workers=6,
    verbose=False
)

In [2]:
# just for debugging

model = Network(9)
model.add_layer(16, ReLU())
model.add_layer(16, ReLU())
model.add_layer(2, Sigmoid())

gs_results = grid_search_cv(
    model, 
    x=X, 
    y=y, 
    n_folds=3,
    metric=MeanEuclideanError(), 
    loss=MeanSquaredError(), 
    eta=[1e-2,],
    nesterov=[0.9,],
    reg_type=[L2],
    reg_val=[1e-7,],
    epochs=300,
    scaler=scaler,
    workers=6
)

Gridsearch: exploring 1 combinations.


100%|██████████| 1/1 [00:26<00:00, 26.20s/it]


In [3]:
results_df = parse_results(gs_results)
results_df.sort_values(by='val_mee', ascending=True)

,eta,nesterov,reg_type,reg_val,tr_mee,val_mee,loss,val_loss
0,0.01,0.9,L2,1.000000e-07,1.451927,1.575261,1.713511,2.048391
